In [1]:
import os
import sys
import pandas
import numpy

import findspark
findspark.init("/usr/local/spark/spark")

import pyspark
from pyspark.sql.window import Window
import pyspark.sql.functions as func

from pyspark.rdd import reduce
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import GaussianMixture, KMeansModel, KMeans
from pyspark.ml.linalg import SparseVector, VectorUDT, Vector, Vectors

In [2]:
from pyspark.sql.functions import udf, col, struct
from pyspark.sql.types import ArrayType, DoubleType, StringType
from pyspark.mllib.linalg.distributed import RowMatrix, DenseMatrix

In [5]:
cluster_files =  "/Users/simondi/PROJECTS/target_infect_x_project/src/tix-analysis/data/outlier-removal"

In [3]:
conf = pyspark.SparkConf().setMaster("local[*]").set("spark.driver.memory", "4G").set("spark.executor.memory", "4G")
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

In [4]:
spark.version

'2.3.0'

In [8]:
from scipy.stats import multivariate_normal

In [28]:
k = 15
svd = multivariate_normal.rvs(mean=scipy.ones(k), size=1000)

In [29]:
dff = map(lambda x: (int(x[0]), Vectors.dense(x[1:])), svd)
df = spark.createDataFrame(dff, schema=["label", "features"])

In [30]:
df.take(1)

[Row(label=0, features=DenseVector([0.4278, -0.3326, -0.3999, 0.1951, 0.3811, 1.4089, 2.3809, 2.0592, -0.9867, 1.1771, 1.9071, 2.0989, -0.803, 2.8963]))]

In [8]:
#df = spark.read.parquet(cluster_files)

In [31]:
for i in range(2, 10 + 1):
    km = GaussianMixture(tol=0.00001, maxIter=1000, k=i, seed=23)
    model = km.fit(df)
    print(i,model.summary.logLikelihood)

2 -19816.073891894677
3 -19682.772876083287
4 -19498.082966786795
5 -19443.46778128294
6 -19303.165846228523
7 -19249.610923261447
8 -19124.65523571511
9 -19013.29310991837
10 -18827.701278437075


In [32]:
for i in range(2, 10 + 1):
    km = GaussianMixture(tol=0.00001, maxIter=1000, k=i, seed=5)
    model = km.fit(df)
    print(i,model.summary.logLikelihood)

2 -19808.771329181203
3 -19690.176802673996
4 -19529.340382725568
5 -19436.789805891458
6 -19287.601381893295
7 -19138.61159520357
8 -19091.268505196804
9 -18948.655596724093
10 -18862.27562835071


In [40]:
for i in range(2, 10 + 1):
    time1 = time.time()
    km = GaussianMixture().setK(i).setSeed(23)
    time2 = time.time()
    model = km.fit(df)
    print(i,model.summary.logLikelihood,time2-time1)

2 -19816.701476302806 0.005852937698364258
3 -19709.27278902305 0.004249095916748047
4 -19550.868034687748 0.004106998443603516
5 -19475.526374945814 0.004667997360229492
6 -19348.387133210228 0.0044231414794921875
7 -19277.555667578006 0.00424504280090332
8 -19146.82639203569 0.003815174102783203
9 -19026.400889437427 0.00701904296875
10 -18887.77810016395 0.004141092300415039


In [35]:
import time

In [39]:
for i in range(2, 10 + 1):
    time1 = time.time()
    km = KMeans().setK(i).setSeed(5)
    time2 = time.time()
    model = km.fit(df)
    print(i, model.computeCost(df), time2-time1)

2 13448.40277863639 0.01022481918334961
3 12922.971721264897 0.00826406478881836
4 12465.260625112052 0.006598234176635742
5 12158.078162664491 0.008065223693847656
6 11872.465213187645 0.006394863128662109
7 11619.028960170446 0.05138707160949707
8 11405.521654965158 0.009996891021728516
9 11243.015674958395 0.0074541568756103516
10 11021.385014091764 0.004087209701538086


In [26]:
for i in range(2, 10 + 1):
    km = KMeans().setK(i).setSeed(2)
    model = km.fit(df)
    print(i, model.computeCost(df))

2 810.0128162664852
3 676.9414245646284
4 550.5300945156876
5 516.2605188186652
6 486.90125896082196
7 492.8494924014178
8 478.907555334699
9 430.9293125715537
10 390.6911177866515


In [27]:
for i in range(2, 10 + 1):
    km = KMeans().setK(i).setSeed(5)
    model = km.fit(df)
    print(i, model.computeCost(df))

2 786.8837213284758
3 629.8876276372729
4 566.9283039950677
5 518.4964089818976
6 480.9845333016365
7 458.5278698446237
8 438.5096785018395
9 407.57614624010097
10 381.34761579730394
